# **Applicant_Data_Clean**

In [1]:
# Step 1: Install required package
!pip install openpyxl

# Step 2: Import libraries
import pandas as pd

# Step 3: Load dataset
df = pd.read_excel("ApplicantData.xlsx")

# ------------------ BEFORE CLEANING ------------------
print("🔎 BEFORE CLEANING")
print("Shape:", df.shape)  # rows, columns
print("\nMissing values per column:\n", df.isnull().sum())
print("\nDuplicate rows:", df.duplicated().sum())
print("\nUnique values in 'Country':\n", df['Country'].unique()[:10])  # show first 10
print("\nUnique values in 'University':\n", df['University'].unique())

# ------------------ CLEANING ------------------
# 1. Drop rows with missing App_ID
df = df.dropna(subset=['App_ID'])

# 2. Remove duplicates
df = df.drop_duplicates()

# 3. Fix Country column (remove emails, keep only alphabetic entries)
df['Country'] = df['Country'].astype(str)
df['Country'] = df['Country'].apply(lambda x: x if x.isalpha() else None)

# 4. Remove rows where Country is still None after cleaning
df = df.dropna(subset=['Country'])

# 5. Standardize text casing
df['Country'] = df['Country'].str.title().str.strip()

# 6. Clean App_ID (remove backticks/spaces)
df['App_ID'] = df['App_ID'].astype(str).str.replace(r"[^A-Za-z0-9]", "", regex=True)

# 7. Standardize phone numbers (keep only digits)
df['Phone_Number'] = df['Phone_Number'].astype(str).str.replace(r"\D", "", regex=True)

# 8. Drop University column if redundant
if df['University'].nunique() == 1:
    df = df.drop(columns=['University'])

# ------------------ AFTER CLEANING ------------------
print("\n✅ AFTER CLEANING")
print("Shape:", df.shape)
print("\nMissing values per column:\n", df.isnull().sum())
print("\nDuplicate rows:", df.duplicated().sum())
print("\nUnique values in 'Country':\n", df['Country'].unique()[:10])

# ------------------ SAVE CLEAN FILE ------------------
df.to_excel("ApplicantData_Clean.xlsx", index=False)

print("\n📂 Cleaned dataset saved as 'ApplicantData_Clean.xlsx'")


🔎 BEFORE CLEANING
Shape: (37882, 4)

Missing values per column:
 App_ID          1
Country         0
University      0
Phone_Number    0
dtype: int64

Duplicate rows: 16489

Unique values in 'Country':
 ['India' 'Nigeria' 'saarthaksingh05@gmail.com' 'satya.sai1881@gmail.com'
 'sharmaishaan16@gmail.com' 'pillir1026@outlook.com'
 'shalinidec05@gmail.com' 'rameshpriyanka536@gmail.com'
 'samjainsamj16@gmail.com' 'ratneshry06@gmail.com']

Unique values in 'University':
 ['Illinois Institute of Technology']

✅ AFTER CLEANING
Shape: (16640, 3)

Missing values per column:
 App_ID          0
Country         0
Phone_Number    0
dtype: int64

Duplicate rows: 949

Unique values in 'Country':
 ['India' 'Nigeria' 'Na' 'Naq' 'Taiwan' 'Pakistan' 'Bangladesh' 'China'
 'Afghanistan' 'Azerbaijan']

📂 Cleaned dataset saved as 'ApplicantData_Clean.xlsx'


# **OutreachData_Before**

In [2]:
import pandas as pd

# Load dataset
df = pd.read_excel("OutreachData.xlsx")

# ----- BEFORE CLEANING -----
before_shape = df.shape
before_missing = df.isnull().sum()
before_duplicates = df.duplicated().sum()
df.to_excel("OutreachData_Before.xlsx", index=False)

# ----- CLEANING -----
df = df.dropna(subset=['Reference_ID'])
df = df.drop_duplicates()
df['Caller_Name'] = df['Caller_Name'].astype(str).str.title().str.strip()
df['Outcome_1'] = df['Outcome_1'].astype(str).str.strip().str.lower()
df['Recieved_At'] = pd.to_datetime(df['Recieved_At'], errors='coerce')
df = df.dropna(subset=['Recieved_At'])
df['Remark'] = df['Remark'].astype(str).str.strip().str.capitalize()
df['University'] = df['University'].astype(str).str.title().str.strip()

# ----- AFTER CLEANING -----
after_shape = df.shape
after_missing = df.isnull().sum()
after_duplicates = df.duplicated().sum()
df.to_excel("outreach_data_clean.xlsx", index=False)

# ----- RESULTS -----
print("Before Cleaning:")
print("Shape:", before_shape)
print("Missing Values:\n", before_missing)
print("Duplicates:", before_duplicates)

print("\nAfter Cleaning:")
print("Shape:", after_shape)
print("Missing Values:\n", after_missing)
print("Duplicates:", after_duplicates)


Before Cleaning:
Shape: (37881, 8)
Missing Values:
 Reference_ID               0
Recieved_At                0
University                 0
Caller_Name                0
Outcome_1                  0
Remark                 33804
Campaign_ID                0
Escalation_Required        0
dtype: int64
Duplicates: 446

After Cleaning:
Shape: (37435, 8)
Missing Values:
 Reference_ID           0
Recieved_At            0
University             0
Caller_Name            0
Outcome_1              0
Remark                 0
Campaign_ID            0
Escalation_Required    0
dtype: int64
Duplicates: 0


# **Campaign_Data_Clean**

In [12]:
import pandas as pd
from io import BytesIO

# Raw data provided by user
data = """ID	Name	Category	Intake	University	Status	Start_Date
AANF23	GR GS FA24 Campaign- Admit, No Deposit	Post Admission	AY2024	Illinois Institute of Technology	Completed	3/20/2024 0:00
AND23	GR GS FA24 Campaign- Deposit No Action	Post Admission	AY2024	Illinois Institute of Technology	Completed	9/11/2024 00:00
BPNANF23	GR GS FA24 Campaign- Deposit, No I-20	Post Admission	AY2024	Illinois Institute of Technology	Completed	7/11/2024 00:00
BPNND23	GR GS FA24 Campaign- In Progress	Pre Admission	AY2024	Illinois Institute of Technology	Completed	3/6/2024 00:00
CTKANF23	GR GS FA24 Campaign- Submit, Incomplete	Pre Admission	AY2024	Illinois Institute of Technology	Completed	3/8/2024 00:00
DANE24	GR GS Call Campaign: India ANF	Pre Admission	AY2024	Illinois Institute of Technology	Completed	4/28/2023 0:00
DNA24	GR GS Call Campaign: India No Deposit	Post Admission	AY2024	Illinois Institute of Technology	Completed	7/3/2023 00:00
FA24AND	GR GS Call Campaign: Other ANF	Post Admission	AY2024	Illinois Institute of Technology	Completed	5/16/2023 0:00
FA24DNA	GR GS Call Campaign: Other No Deposit	Post Admission	AY2024	Illinois Institute of Technology	Completed	7/5/2023 00:00
FA24DNI	GR GS SP25 Campaign- All I-20s Sent	Post Admission	AY2024	Illinois Institute of Technology	Completed	12/5/2024 00:00
FA24IP	GR GS SP25 Campaign- Admit, No Deposit	Post Admission	AY2024	Illinois Institute of Technology	Completed	10/9/2024 00:00
FA24SIC	GR GS SP25 Campaign- Deposit, No I-20	Post Admission	AY2024	Illinois Institute of Technology	Completed	10/22/2024 0:00
IANF23	GR GS SP25 Campaign- Deferrals to SP25	Post Admission	AY2024	Illinois Institute of Technology	Completed	11/5/2024 00:00
IND23	GR GS SP25 Campaign- In Progress	Pre Admission	AY2024	Illinois Institute of Technology	Completed	10/7/2024 00:00
OANF23	GR GS SP25 Campaign- New Inquiry	Pre Admission	AY2024	Illinois Institute of Technology	Completed	10/18/2024 0:00
OND23	GR GS SP25 Campaign- Submitted, Incomplete	Pre Admission	AY2024	Illinois Institute of Technology	Completed	10/9/2024 0:00
SP25AI2S	GR GS Call Campaign: Africa ANF	Pre Admission	AY2024	Illinois Institute of Technology	Completed	5/18/2023 0:00
SP25AND	GR GS Call Campaign: Africa No Deposit	Post Admission	AY2024	Illinois Institute of Technology	Completed	7/5/2023 00:00
SP25DN1	GR GS Call Campaign: Bangladesh, Pakistan, Nepal ANF	Pre Admission	AY2024	Illinois Institute of Technology	Completed	5/16/2023 0:00
SP25DSP	GR GS Call Campaign: Bangladesh, Pakistan, Nepal No Deposit	Post Admission	AY2024	Illinois Institute of Technology	Completed	7/5/2023 00:00
SP25IP	GR GS Call Campaign: China, Taiwan, Korea ANF	Pre Admission	AY2024	Illinois Institute of Technology	Completed	5/16/2023 0:00
SP25NIQ	Deposit and Advised, Not Enrolled	Post Admission	AY2024	Illinois Institute of Technology	Completed	7/17/2024 0:00
SP25SIC	Deposit, Not Advised	Post Admission	AY2024	Illinois Institute of Technology	Completed	7/17/2024 0:00"""

# Load into DataFrame
from io import StringIO
df = pd.read_csv(StringIO(data), sep="\t")

# Cleaning process
df = df.dropna(subset=['ID', 'Name', 'Start_Date'])
df = df.drop_duplicates()
df['Category'] = df['Category'].astype(str).str.title().str.strip()
df['University'] = df['University'].astype(str).str.title().str.strip()
df['Status'] = df['Status'].astype(str).str.lower().str.strip()
df['Start_Date'] = pd.to_datetime(df['Start_Date'], errors='coerce')
df = df.dropna(subset=['Start_Date'])

# Save to Excel in memory
output = BytesIO()
df.to_excel(output, index=False, engine='openpyxl')
output.seek(0)


0

In [13]:
df.to_excel("campaign_data_clean.xlsx", index=False)